In [30]:
# install fast_ml library
!pip install langchain-openai --quiet
!pip install typing_extensions==4.10.0 --upgrade

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 7, 35, Finished, Available, Finished)

In [13]:

# imports
import pandas as pd
import re
from azureml.core import Workspace, Dataset
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 18, Finished, Available, Finished)

In [4]:
# Obtain these details from the Data Assets
subscription_id = ''
resource_group = ''
workspace_name = ''

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = workspace.get_default_datastore()

dataset = Dataset.get_by_name(workspace, name='filename') #replace filename
df = dataset.to_pandas_dataframe()

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 9, Finished, Available, Finished)

In [6]:
# preview data
df.head()

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 11, Finished, Available, Finished)

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,cnt,target,description
0,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,985,2011-01-01,Today the temperature was 1 with a level of hu...
1,2,2011-01-02,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,801,2011-01-02,Today the temperature was 1 with a level of hu...
2,3,2011-01-03,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349,2011-01-03,Today the temperature was 1 with a level of hu...
3,4,2011-01-04,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,1562,2011-01-04,Today the temperature was 1 with a level of hu...
4,5,2011-01-05,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,1600,2011-01-05,Today the temperature was 1 with a level of hu...


In [7]:
# Initialize the OpenAI API key and endpoint for Azure OpenAI
openai_key = ""
openai_endpoint = ""
openai_deployment_name = "OpenAI-Model-RND-GPT4"
openai_api_version = "2024-02-01" 

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 12, Finished, Available, Finished)

In [119]:
class ChatHandler:
    def __init__(self) -> None:
        self.llm = AzureChatOpenAI(
            azure_deployment=openai_deployment_name,
            api_key = openai_key,
            api_version = openai_api_version,
            azure_endpoint = openai_endpoint
        )

    def categorise_review(self, text):
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    f"You are a helpful assistant that categorises this \"{text}\ into keywords.",
                ),
                ("human", "{input}"),
            ]
        )

        chain = prompt | self.llm
        response = chain.invoke(
            {
                "input": text,
            }
        )

        # Extract text content from the AIMessage object
        content = response.content

        # Split content
        lines = content.split('\n')

        # Strip whitespace around each keyword
        keywords = [line.lstrip('- ').strip() for line in lines if line.strip()]

        return keywords

    def categorise_reviews_in_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        # Ensure the 'description' column exists
        if 'description' not in df.columns:
            raise ValueError("The DataFrame must contain a 'description' column")

        # Categorise each review and store the result in a new column
        df['category'] = df['description'].apply(self.categorise_review)

        return df

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 124, Finished, Available, Finished)

In [124]:
df1 = df.head(1)

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 129, Finished, Available, Finished)

In [125]:
chat_handler = ChatHandler()

# Call the method to categorize reviews in the DataFrame
categorized_df = chat_handler.categorise_reviews_in_dataframe(df1)

# Display the updated DataFrame
categorized_df.head()

StatementMeta(801348e3-34b6-4071-9673-1a9df40fd4d2, 8, 130, Finished, Available, Finished)

/tmp/ipykernel_23135/4062343107.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['description'].apply(self.categorise_review)


,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,cnt,target,description,category
0,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,985,2011-01-01,Today the temperature was 1 with a level of humidity equal to 2,"[Temperature, 1 (degree), Humidity, 2 (percentage or relative value, context needed), Weather conditions, Climate data]"
